# This notebook describes the development of a set of functions seeking to numerically determine the optimal number of quadratic voting credits '*C*' given how many distinct items '*N*' are being voted on.
- **In sum, given a quadratic ballot with N issues, how many voting credits should be given?**

## 1.  This was done to address one of the strongest criticisms of Quadratic Voting (QV), wasted voting power (vote credits that cannot be used without changing the voting intention of the voter).  This is particularly true of small N ballots.

#### *Note: Another way of doing QV is through continuous, or "infinitely divisible" votes.  Thus, a voter can use any N real numbers that when squared and added together equal C.  While this solves the wasted credit issue, there are still game theory implications to different levels of C.  This is a future path of investigation.*

## 2.  As mentioned in the note above, there are optimal strategies from both a game theory and computational front.

## 3.  Limitations:  This notebook is limited to discrete, positive integer votes.  It does not take into account negative votes for a ballot item like many QV frameworks do.  Having negative votes will likely alter the method of selecting strategies and is another path of investigation.  Also, perfectly rational voters are asumed - thus, while the model only uses rational strategies in its mechanics, in reality the voters will have the full set of possibilities to choose from, including ones where they have a better strategy than what their voting is, or making suboptimal tradeoffs.  This should be irrelevant, as strategy sets should be judged on rational voting.

----

#### Libraries Used

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
import math
from itertools import combinations_with_replacement, groupby
from math import floor
import re

----

## Key Terms and Concepts Explored Below
1. QUADRATIC VOTING (QV): Voting method where voters are given a number of voting credits 'C' and allowed to distribute them across a series of N-items.  The credits convert to tallied votes at a rate of $\sqrt(C)$ per voting item.

2. DECISION SPACE: The collection of N-items being quadratically voted on, where N is a positive integer.  Basically a ballot.

3. VOTING ITEM: One of N-items (voting choices) that a voter can spend voting power on.

4. VOTER: An individual who choses a voting strategy by assinging voting power, quadratically, across voting items within an N-sized decision space.

5. VOTE: How much voting power is ultiamtely assigned to a Voting Item given the voter's strategy. $V = \sqrt(C)$

6. VOTE CREDIT: 'C' units of voting power which convert into votes quadratically.  The goal of this notebook is to find optimal values of C given N.

7. STRATEGY: The set of numbers which represent how a voter voted using their voting power.  For example, when N = 4 & C = 50, one voting strategy that wastes no credits would be: [7,1,0,0].  Strategies go from highest vote count on the left to lowest on the right as each strategy represents all equivalent voting strategies (i.e. [4,2,1,0,0] == [2,4,0,0,1]

8. MAX VOTE: 'Vmax' = The maximum possible vote on a single voting item for a given C.  Always lower than total votes from a given strategy, unless C is a perfect square and all voting power is used on a single voting item; in that case they are equal. 

9. NULL STRATEGY: A voting strategy where all votes are equivalent, ie. [2,2,2], [4,4,4,4,4,4], etc..  These are all equivalnet to [0,..,0], aka not voting.

10. STRATEGY SET: The collection of strategies available for a given set of parameters (N and m or C).  Range from "Complete" (all non-null strategies, with Equivalent Strategies consolidated to one) to "Reduced Form".

10. REDUCED FROM (OF STRATEGY SET): The set of strategies that use the minimal number of votes to express a group of voters relative preferences.

11. EQUIVALENT STRATEGY: Equivalent strategies are all permutations of the same numbers within the voting space.  As such they combined into one for the analysis.  For example strategy [2,2,1] == [2,1,2] == [1,2,2].

## Assumptions
1. All null strategies are equivalent.
2. All equivalent strategies are equivalent.
3. N = 1 decouples into N = 2
4. For N = 2, there is only one strategy, [1,0].  Therefore C(2) = 1.
5. Voters are rational.  They will only waste voting credits if it fits their preferred strategy, and strategies suffer a penalty for wasting votes (waste factor).
6. Voters will always seek to maximize their largest vote element rather than waste votes.  
7. Voters most concerned about the fairness of their vote, not the absolute.

## Process Steps 
1. Create the complete strategy set given N and a number of voice credits C (include strategies).
2. Reduce the strategy set to rational strategies (reduced form).
3. Determine key metrics for strategy sets of different values of C.
4. Compare all strategy sets for a given N to determine which value of C is optimal (allows for comprehensive voting actions and equitable strategies).

----

## STEP 1: Create the sets of unique voting strategies for a given N-sized decision space bounded by different numbers of voice credits, C.

The method used first assigns a value 'm' to represent the highest number of votes a voter can apply to a single voting item.  

For example, the full solution set of strategies for [N = 3, m = 2] would be:
[2,2,2], [2,2,1], [2,1,1], [2,1,0], [2,0,0], [1,1,1], [1,1,0], [1,0,0], [0,0,0]

    Note: [2,2,1] == [2,1,2] == [1,2,2], where [2,2,1] is the 'representative form'

Also, all null strategies are dropped, thus leaving the "null-free" solution set as:
[2,2,1], [2,1,1], [2,1,0], [2,0,0], [1,1,0], [1,0,0]

### 1a. Build all strategies for a given N up to the strategy set [m,m,...,m-1] .  
    Note: The strategy set [m,m,...,m] results in the null strategy, so it is not included as all null strategies should be removed.  Also, equivalent strategies should be combined into one.

In [2]:
def strategies_for_n_given_m(m, N):
    '''
    Returns possible voting strategies, along with the total votes given by the strategies and the strategy's
    total cost in credits.
    
    Parameters:
        int m = Top limit/max number of votes for a voting item
        int N = Number of Issues
        
    Returns:
        obj output = List of dicts showing complete strategy set for N given m, with total votes and credits calculated.         
    '''
    # Create an iterable for all values from 0 to m:
    numbers = list(range(m + 1))
    
    # Generate all unique combinations of length N from the numbers 0 to m
    combs = list(combinations_with_replacement(numbers, N))
    
    # Remove equivalent strategies (e.g. [2,1] == [1,2]).  !-set() looks at both numbers:
    combs = list(set(c for c in combs))
    
    # Sort each list in descending order - descending because the convention is to list them high to low
    combs = sorted(combs, key=lambda x: x[::-1], reverse = True)
    
    # Create return object - a list of dicts for easy df creation
    output = []
    
    # Get the vote credit total needed for each strategy
    for i in combs:
        # FIRST FILTER: Remove all strategies where the elements are equal (equilvalent to Null Vote)
        if i == i[0:1] * N:
            #print(f'Voting strategy {i} == Null Vote.  Removed.')
            continue
            
        # If not a null strategy, build an entry for output
        else:    
            i = sorted(i, reverse = True)
            
            # Create an object to hold the strategy and its total votes and vote_credits needed
            output_dict = {}
            output_dict['strategy'] = i
            
            output_dict['vote_sum'] = sum(i)
            output_dict['vote_credits'] = sum([p**2 for p in i])
            
            output.append(output_dict) 
    
    return output

In [3]:
complete_strats = strategies_for_n_given_m(4, 4)
complete_strats, len(complete_strats)

([{'strategy': [4, 4, 4, 3], 'vote_sum': 15, 'vote_credits': 57},
  {'strategy': [4, 4, 4, 2], 'vote_sum': 14, 'vote_credits': 52},
  {'strategy': [4, 4, 4, 1], 'vote_sum': 13, 'vote_credits': 49},
  {'strategy': [4, 4, 4, 0], 'vote_sum': 12, 'vote_credits': 48},
  {'strategy': [4, 4, 3, 3], 'vote_sum': 14, 'vote_credits': 50},
  {'strategy': [4, 4, 3, 2], 'vote_sum': 13, 'vote_credits': 45},
  {'strategy': [4, 4, 3, 1], 'vote_sum': 12, 'vote_credits': 42},
  {'strategy': [4, 4, 3, 0], 'vote_sum': 11, 'vote_credits': 41},
  {'strategy': [4, 4, 2, 2], 'vote_sum': 12, 'vote_credits': 40},
  {'strategy': [4, 4, 2, 1], 'vote_sum': 11, 'vote_credits': 37},
  {'strategy': [4, 4, 2, 0], 'vote_sum': 10, 'vote_credits': 36},
  {'strategy': [4, 4, 1, 1], 'vote_sum': 10, 'vote_credits': 34},
  {'strategy': [4, 4, 1, 0], 'vote_sum': 9, 'vote_credits': 33},
  {'strategy': [4, 4, 0, 0], 'vote_sum': 8, 'vote_credits': 32},
  {'strategy': [4, 3, 3, 3], 'vote_sum': 13, 'vote_credits': 43},
  {'strategy

> For N = 4 and m = 4, there are 65 null-less strategies representing a range of voting credits needed as well as some additional information.

#### In general, the formula for determining the number of non-null strategies given m (where m = positive integer) and n (n>1) is:

    s = ((m+1) * (m+2) * ... * (m+n)) / n!

#### And given m (where m is both a positive and negative integer):

    s = ((2m + 1) * (2m + 2) * ... * (2m+n)) / n!

### 1b. Build the strategy set by creating m from vote credits and filtering out all stategies that do not work with the given vote credit amount.

In [4]:
def build_strategy_set(N, vote_credits):
    '''
    Use vote credits to filter out stragies from a complete N,m strategy set.
        
    Parameters:
        int N = Number of Issues
        int vote_credits = C = Maximum amount of vote credits to get N-length strategies for
        
    Returns:
        obj s_set = Dataframe with all possible strategies along with their vote totals and credit requirements up to C. 
    '''
    # First identify the value of m, which is the highest possible value for an individual Voting Item's total.
    m = floor(vote_credits**.5)
   
    # Now create and filter out all strategies with C greater than given vote_credits
    s_set = pd.DataFrame(strategies_for_n_given_m(m,N))
    s_set = s_set[s_set['vote_credits'] <= vote_credits]
    s_set = s_set.reset_index(drop = True)
    
    return s_set

In [5]:
build_strategy_set(4, 50)

,strategy,vote_sum,vote_credits
0,"[7, 1, 0, 0]",8,50
1,"[7, 0, 0, 0]",7,49
2,"[6, 3, 2, 1]",12,50
3,"[6, 3, 2, 0]",11,49
4,"[6, 3, 1, 1]",11,47
...,...,...,...
102,"[2, 1, 0, 0]",3,5
103,"[2, 0, 0, 0]",2,4
104,"[1, 1, 1, 0]",3,3
105,"[1, 1, 0, 0]",2,2


## STEP 2: Reduce the strategy set to rational strategies (reduced form).

There are two reductions that need to happen to reduce to rational strategies (that is strategies that aren't equivalent):
1. 'Arithmatic': Where each strategy element is reduced by one until at least one element is zero. *For +/-m, reduction means get 1 integer closer to zero.  The non-reduced strategy is eliminated.
    Ex. [4,3,2,2] ~ [3,2,1,1] ~ [2,1,0,0]
    
2. 'Multiplicative': Where each strategy element is divided by a number such that the result for each is an integer.
    Ex. [8,6,6,6,4] ~ [4,3,3,3,2]; Note: 0 divided by any number is an integer.  The reduced strategy is eliminated.
    
Both reduce the strategy set by removing arithmatically equivalent strategies (different than equivalent strategies above).

### 2a. Create a subfunction to reduce to lowest form, that means the smallest equivalent voting pattern (made by subtracting one from each vote until there is a zero column).

In [6]:
def arithmatic_reduction(strategy_list):
    '''
    Reduce strategy set by going through strategies and seeing if each can be reduced, and if so appending
    a return list with that rather than the original strategy.
    
    Parameters:
        obj df = Df with strategies
        
    Returns:
        obj new_strat_list = List of strategies after arithatic reduction
    '''
    # Create the holder for the output
    new_strat_list = []
    # Create the holder for strategies to remove
    remove_strats = []
    
    #strategy_list = df.strategy

    # Test each strategy to find cases where the elements are 1 vote off, and can therefore be removed.
    for strategy in strategy_list:
        # Break swith breaks out of loop
        mod_strategy = []
        for element in strategy:
            new_element = element - 1
            mod_strategy.append(new_element)
        
        remove_strats.append(mod_strategy)
        
    for strategy in strategy_list:
        if strategy in remove_strats:
            continue
        else:
            new_strat_list.append(strategy)
        
    return new_strat_list

In [7]:
arithmatic_reduction(build_strategy_set(4, 50).strategy)

[[7, 1, 0, 0],
 [7, 0, 0, 0],
 [6, 3, 2, 1],
 [6, 3, 2, 0],
 [6, 3, 1, 1],
 [6, 3, 1, 0],
 [6, 3, 0, 0],
 [6, 2, 2, 2],
 [6, 2, 2, 1],
 [6, 2, 2, 0],
 [6, 2, 1, 1],
 [6, 2, 1, 0],
 [6, 2, 0, 0],
 [6, 1, 1, 1],
 [6, 1, 1, 0],
 [6, 1, 0, 0],
 [6, 0, 0, 0],
 [5, 5, 0, 0],
 [5, 4, 3, 0],
 [5, 4, 2, 2],
 [5, 4, 2, 1],
 [5, 4, 2, 0],
 [5, 4, 1, 1],
 [5, 4, 1, 0],
 [5, 4, 0, 0],
 [5, 3, 3, 2],
 [5, 3, 3, 1],
 [5, 3, 3, 0],
 [5, 3, 2, 2],
 [5, 3, 2, 1],
 [5, 3, 2, 0],
 [5, 3, 1, 1],
 [5, 3, 1, 0],
 [5, 3, 0, 0],
 [5, 2, 2, 2],
 [5, 2, 2, 1],
 [5, 2, 2, 0],
 [5, 2, 1, 1],
 [4, 4, 4, 1],
 [4, 4, 4, 0],
 [4, 4, 3, 3],
 [4, 4, 3, 2],
 [4, 4, 3, 1],
 [4, 4, 3, 0],
 [4, 4, 2, 2],
 [4, 4, 2, 1],
 [4, 4, 2, 0],
 [4, 4, 1, 1],
 [4, 4, 1, 0],
 [4, 4, 0, 0],
 [4, 3, 3, 3],
 [4, 3, 3, 2],
 [4, 3, 3, 1],
 [4, 3, 3, 0],
 [4, 3, 2, 2],
 [4, 3, 2, 1],
 [4, 3, 2, 0],
 [4, 2, 2, 2],
 [3, 3, 3, 2],
 [3, 3, 3, 1]]

### 2c. Second reduction, to be applied in larger function.

In [8]:
def remove_similar(strategy_list):
    '''
    This is the multiplicative reduction.  It analyzes each as a vector to see if they have the same unit vector, 
    meaning they are scalar multiples of one another.  It removes the duplicate vector with the lower magnitude.
    
    Parameters:
        obj strategy_list = List of strategies
        
    Returns:
        obj unique_vectors = List of multiplicatively reduced strategies
    '''
    unique_vectors = []

    for v in strategy_list:
        unit_vector = v / LA.norm(v)
        is_unique = True
        for uv in unique_vectors:
            uv_unit_vector = uv / LA.norm(uv)
            if all(unit_vector == uv_unit_vector):
                if LA.norm(uv) < LA.norm(v):
                    unique_vectors.remove(uv)
                    unique_vectors.append(v) 
                    unique_vectors.remove(v)

                is_unique = False
                break
        if is_unique:
            unique_vectors.append(v)
            
    return unique_vectors

In [9]:
remove_similar(build_strategy_set(4, 50).strategy)

[[7, 1, 0, 0],
 [7, 0, 0, 0],
 [6, 3, 2, 1],
 [6, 3, 2, 0],
 [6, 3, 1, 1],
 [6, 3, 1, 0],
 [6, 3, 0, 0],
 [6, 2, 2, 2],
 [6, 2, 2, 1],
 [6, 2, 2, 0],
 [6, 2, 1, 1],
 [6, 2, 1, 0],
 [6, 2, 0, 0],
 [6, 1, 1, 1],
 [6, 1, 1, 0],
 [6, 1, 0, 0],
 [5, 5, 0, 0],
 [5, 4, 3, 0],
 [5, 4, 2, 2],
 [5, 4, 2, 1],
 [5, 4, 2, 0],
 [5, 4, 1, 1],
 [5, 4, 1, 0],
 [5, 4, 0, 0],
 [5, 3, 3, 2],
 [5, 3, 3, 1],
 [5, 3, 3, 0],
 [5, 3, 2, 2],
 [5, 3, 2, 1],
 [5, 3, 2, 0],
 [5, 3, 1, 1],
 [5, 3, 1, 0],
 [5, 3, 0, 0],
 [5, 2, 2, 2],
 [5, 2, 2, 1],
 [5, 2, 2, 0],
 [5, 2, 1, 1],
 [5, 2, 1, 0],
 [5, 2, 0, 0],
 [5, 1, 1, 1],
 [5, 1, 1, 0],
 [5, 1, 0, 0],
 [4, 4, 4, 1],
 [4, 4, 4, 0],
 [4, 4, 3, 3],
 [4, 4, 3, 2],
 [4, 4, 3, 1],
 [4, 4, 3, 0],
 [4, 4, 2, 2],
 [4, 4, 2, 1],
 [4, 4, 2, 0],
 [4, 4, 1, 1],
 [4, 4, 1, 0],
 [4, 3, 3, 3],
 [4, 3, 3, 2],
 [4, 3, 3, 1],
 [4, 3, 3, 0],
 [4, 3, 2, 2],
 [4, 3, 2, 1],
 [4, 3, 2, 0],
 [4, 3, 1, 1],
 [4, 3, 1, 0],
 [4, 3, 0, 0],
 [4, 2, 2, 2],
 [4, 2, 2, 1],
 [4, 2, 2, 0],
 [4, 2, 1,

### 2d. Create function that determines inefficent statategies and then removes them, finding the 'cutoff' for the inefficiency.  For example, if you have [4,1,0] and you can vote [5,1,0] and waste less credits, you do the latter.  The amount of wasted credits in [4,1,0] becomes the maximum + 1 wasted credit limit.

In [10]:
def get_cutoff(strategies):
    '''
    Finds the highest vote total strategy which is inefficient, meaning there is a better voting strategy.
    For example, if a voter has both [5,1,0] and [4,1,0] available to them, the would always rationally go with
    the [5,1,0], as one of the principles to voting is its less about ratios since all are voting items are 
    independant except for the decision itself.  Thus, the case for this is where all but the max vote total
    are the same between two strategies, and the max vote of one is higher.
    
    Parameters:
        obj strategies = List of strategies
        
    Returns:
        str = List in string form of cutoff strategy
    '''
    strategies.reverse()
    for item_a in strategies:
        for item_b in strategies:
            if [a - b for a, b in zip(item_a, item_b)] == [1] + [0] * (len(item_a)-1):
                if item_b[0] != item_b[1]:
                    return str(item_b)

In [11]:
get_cutoff(list(build_strategy_set(4, 50).strategy))

'[1, 0, 0, 0]'

### 2e. Lastly create the difference analysis, which creates the metrics for the strategy sets to be compared to one another

In [12]:
def vote_difference_analysis(strategy):
    '''
    Outputs some comparison metrics about the individual voting items within a strategy.
    
    Parameters:
        obj strategy = List of integers
        
    Returns:
        int max_diff = Minimum difference of votes across voting elements
        int min_diff = Maximum difference of votes across voting elements
        int max_min_diff = Difference between max and min vote differences
        float avg_diff = Average of differences of votes across voting elements
    '''
    # Create a list to hold differences between votes for strategy
    res = []
    
    for i in range(len(strategy)):
        for j in range(len(strategy)):
            if i != j:
                res.append(abs(strategy[i] - strategy[j]))
    
    # Get vote differences
    max_diff = max(res)
    min_diff = min(res)
    max_min_diff = max_diff - min_diff
    avg_diff = round(sum(res)/len(res), 4)
    
    return max_diff, min_diff, max_min_diff, avg_diff

## STEP 3: Determine key metrics for strategy sets of different values of C, given N

In [13]:
'Determine key metrics for strategy sets of different values of C, given N'.upper()

'DETERMINE KEY METRICS FOR STRATEGY SETS OF DIFFERENT VALUES OF C, GIVEN N'

In [14]:
def strategies_for_n(N, max_credits):
    '''
    Given a number of max credits, determine all viable strategy options for N.
    Basically: set max_credits, run the function, then pick lowest C that optimizes the voting pattern.
    
    Parameters:
        int N = Number of Issues
        int max_credits = Cmax = Maximum amount of vote credits to get N-length strategies for
        
    Returns:
        obj credit_outcomes = Dataframe with details on strategy sets for each C in N up to Cmax
    '''
    strategies = build_strategy_set(N, max_credits).sort_values(by = 'vote_credits')
    
    credit_outcomes = []
    
    for i in range(1, max_credits + 1):
        
        print('Voice Credits:',i)
        
        # Vote max for the given i
        vmax = floor(i**.5)
        
        # Create list object to hold dicts ceated from strategy details
        result = []        
        
        # Extract the strategies that work with this number of votes
        strats_in_i = strategies[strategies['vote_credits'] <= i]
        strat_list = list(strats_in_i.strategy)
    
        strat_list = arithmatic_reduction(strat_list)
        
        # You can now remove the similar strategies having removed any ones higher that would skew this output
        strat_list = remove_similar(strat_list)

        strats_in_i = strats_in_i[strats_in_i.strategy.isin(strat_list)]
        
        # Create the metrics for each strategy
        for row in strats_in_i.strategy:
            # Basic info
            output = {}
            output['strategy'] = row
            row_votes = sum(row)
            row_vc = sum([x**2 for x in row])
            output['votes per credit'] = row_votes/row_vc

            output['wasted_credits'] = i - row_vc
            
            # Difference metrics
            max_diff, min_diff, max_min_diff, avg_diff = vote_difference_analysis(row)
                
            output['min_vote_diff'] = min_diff
            output['max_vote_diff'] = max_diff
            output['max_min_diff'] = max_min_diff
            output['avg_diff'] = avg_diff
            
            # Scoring metrics
            output['votes_over_vmax'] = row_votes - vmax
            output['credits_over_vmax'] = row_vc - vmax**2
            
            # Create a weight factor for each strategy 
            # This will weight each strategy's effect on the strategy set of a given C
            # !- vary wf below 
            wf = .5
            output['WF1'] = wf**(output['wasted_credits'])
            result.append(output)
        
        result = pd.DataFrame(result)
        
        # Make a list of strategies available for N
        n_strategies = list(result.strategy)

        # Identify a 'cutoff' strategy = last filter
        cutoff_strategy = get_cutoff(n_strategies)
        print('cutoff_strategy',cutoff_strategy)

        # Need to change strategy back into a string
        result = result.astype({'strategy':str})

        # Find cutoff strategy and how many credits it wastes
        wasted_credits_cutoff = result[result['strategy'] == cutoff_strategy]['wasted_credits'].values
        print('wasted_credits_cutoff',wasted_credits_cutoff)

        # Remove all strategies which waste more credits than the cutoff (including the cutoff)
        if len(wasted_credits_cutoff) > 0:
            result = result[result['wasted_credits'] < wasted_credits_cutoff[0]]
            
        # Create additional score columns using weight factor
        result['weighted_score'] = (result['votes_over_vmax'] + result['credits_over_vmax']) * result['WF1']
        
        ## ## ## WEIGHTED DIFFERENCES
        result['min_vote_diff'] = result['min_vote_diff'] * result['WF1']
        result['max_vote_diff'] = result['max_vote_diff'] * result['WF1']
        result['max_min_diff'] = result['max_min_diff'] * result['WF1']
        result['avg_diff'] = result['avg_diff'] * result['WF1']
        ## ## ##
         
        # Create Strategy Set metrics for a given C
        sc = len(result) # Count of strategies
        v_c = result['votes per credit'].mean() 
        wc_s = result['wasted_credits'].sum() # Sum of wasted votes
        wc_m = result['wasted_credits'].mean() # Average of wasted credits for all strategies
        min_vd_m = result['min_vote_diff'].mean()
        max_vd_m = result['max_vote_diff'].mean()
        d_vd_m = result['max_min_diff'].mean()
        avg_diff_m = result['avg_diff'].mean()
        score_s = result['weighted_score'].sum()
        score_m = result['weighted_score'].mean()
        wf_s = result['WF1'].sum()
        wf_m = result['WF1'].mean()
        
        credit_outcomes.append({'Credits':i,
                                'Strategies':sc,
                                'Votes per Credit':v_c,
                                'Total Wasted Credits':wc_s,
                                'Avg. Wasted Credits per Strategy':wc_m,
                                'Avg. Min Vote Diff':min_vd_m,
                                'Avg. Max Vote Diff':max_vd_m,
                                'Avg. MinMax Vote Diff':d_vd_m,
                                'Avg. Diff':avg_diff_m,
                                'Avg. Score':score_m,
                                'Sum of Scores':score_s,
                                'Avg. Weight':wf_m,
                                'Sum of Weights':wf_s})
      
        #display(result) ### To remove
    
    return credit_outcomes

In [15]:
res = strategies_for_n(4, 50)

Voice Credits: 1
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 2
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 3
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 4
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 5
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 6
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 7
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 8
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 9
cutoff_strategy None
wasted_credits_cutoff []
Voice Credits: 10
cutoff_strategy [2, 1, 0, 0]
wasted_credits_cutoff [5]
Voice Credits: 11
cutoff_strategy [2, 1, 1, 0]
wasted_credits_cutoff [5]
Voice Credits: 12
cutoff_strategy [2, 1, 1, 1]
wasted_credits_cutoff [5]
Voice Credits: 13
cutoff_strategy [2, 1, 1, 1]
wasted_credits_cutoff [6]
Voice Credits: 14
cutoff_strategy [2, 1, 1, 1]
wasted_credits_cutoff [7]
Voice Credits: 15
cutoff_strategy [2, 1, 1, 1]
wasted_credits_cutoff

#### .py Test:

In [16]:
from qv_credit_optimization import strategies_for_n

In [17]:
strategies_for_n(3,26, display_details = True)

Voice Credits: 1
cutoff_strategy None
wasted_credits_cutoff []


,strategy,votes per credit,wasted_credits,min_vote_diff,max_vote_diff,max_min_diff,avg_diff,votes_over_vmax,credits_over_vmax,WF1,weighted_score
0,"[1, 0, 0]",1.0,0,0.0,1.0,1.0,0.6667,0,0,1.0,0.0


Voice Credits: 2
cutoff_strategy None
wasted_credits_cutoff []


,strategy,votes per credit,wasted_credits,min_vote_diff,max_vote_diff,max_min_diff,avg_diff,votes_over_vmax,credits_over_vmax,WF1,weighted_score
0,"[1, 0, 0]",1.0,1,0.0,0.5,0.5,0.33335,0,0,0.5,0.0
1,"[1, 1, 0]",1.0,0,0.0,1.0,1.0,0.66670,1,1,1.0,2.0


Voice Credits: 3
cutoff_strategy None
wasted_credits_cutoff []


...

Voice Credits: 25
cutoff_strategy [3, 2, 2]
wasted_credits_cutoff [8]


,strategy,votes per credit,wasted_credits,min_vote_diff,max_vote_diff,max_min_diff,avg_diff,votes_over_vmax,credits_over_vmax,WF1,weighted_score
4,"[3, 3, 0]",0.333333,7,0.0000,0.023438,0.023438,0.015625,1,-7,0.007812,-0.046875
5,"[4, 1, 1]",0.333333,7,0.0000,0.023438,0.023438,0.015625,1,-7,0.007812,-0.046875
6,"[3, 3, 1]",0.368421,6,0.0000,0.031250,0.031250,0.020833,2,-6,0.015625,-0.062500
7,"[4, 2, 0]",0.300000,5,0.0625,0.125000,0.062500,0.083334,1,-5,0.031250,-0.125000
8,"[4, 2, 1]",0.333333,4,0.0625,0.187500,0.125000,0.125000,2,-4,0.062500,-0.125000
9,"[3, 3, 2]",0.363636,3,0.0000,0.125000,0.125000,0.083337,3,-3,0.125000,0.000000
10,"[4, 2, 2]",0.333333,1,0.0000,1.000000,1.000000,0.666650,3,-1,0.500000,1.000000
11,"[4, 3, 0]",0.280000,0,1.0000,4.000000,3.000000,2.666700,2,0,1.000000,2.000000


Voice Credits: 26
cutoff_strategy [4, 1, 0]
wasted_credits_cutoff [9]


,strategy,votes per credit,wasted_credits,min_vote_diff,max_vote_diff,max_min_diff,avg_diff,votes_over_vmax,credits_over_vmax,WF1,weighted_score
3,"[3, 3, 0]",0.333333,8,0.00000,0.011719,0.011719,0.007812,1,-7,0.003906,-0.023438
4,"[4, 1, 1]",0.333333,8,0.00000,0.011719,0.011719,0.007812,1,-7,0.003906,-0.023438
5,"[3, 3, 1]",0.368421,7,0.00000,0.015625,0.015625,0.010416,2,-6,0.007812,-0.031250
6,"[4, 2, 0]",0.300000,6,0.03125,0.062500,0.031250,0.041667,1,-5,0.015625,-0.062500
7,"[4, 2, 1]",0.333333,5,0.03125,0.093750,0.062500,0.062500,2,-4,0.031250,-0.062500
8,"[3, 3, 2]",0.363636,4,0.00000,0.062500,0.062500,0.041669,3,-3,0.062500,0.000000
9,"[4, 2, 2]",0.333333,2,0.00000,0.500000,0.500000,0.333325,3,-1,0.250000,0.500000
10,"[4, 3, 0]",0.280000,1,0.50000,2.000000,1.500000,1.333350,2,0,0.500000,1.000000
11,"[4, 3, 1]",0.307692,0,1.00000,3.000000,2.000000,2.000000,3,1,1.000000,4.000000
12,"[5, 1, 0]",0.230769,0,1.00000,5.000000,4.000000,3.333300,1,1,1.000000,2.000000


[{'Credits': 1,
  'Strategies': 1,
  'Votes per Credit': 1.0,
  'Total Wasted Credits': 0,
  'Avg. Wasted Credits per Strategy': 0.0,
  'Avg. Min Vote Diff': 0.0,
  'Avg. Max Vote Diff': 1.0,
  'Avg. MinMax Vote Diff': 1.0,
  'Avg. Diff': 0.6667,
  'Avg. Score': 0.0,
  'Sum of Scores': 0.0,
  'Avg. Weight': 1.0,
  'Sum of Weights': 1.0},
 {'Credits': 2,
  'Strategies': 2,
  'Votes per Credit': 1.0,
  'Total Wasted Credits': 1,
  'Avg. Wasted Credits per Strategy': 0.5,
  'Avg. Min Vote Diff': 0.0,
  'Avg. Max Vote Diff': 0.75,
  'Avg. MinMax Vote Diff': 0.75,
  'Avg. Diff': 0.5000249999999999,
  'Avg. Score': 1.0,
  'Sum of Scores': 2.0,
  'Avg. Weight': 0.75,
  'Sum of Weights': 1.5},
 {'Credits': 3,
  'Strategies': 2,
  'Votes per Credit': 1.0,
  'Total Wasted Credits': 3,
  'Avg. Wasted Credits per Strategy': 1.5,
  'Avg. Min Vote Diff': 0.0,
  'Avg. Max Vote Diff': 0.375,
  'Avg. MinMax Vote Diff': 0.375,
  'Avg. Diff': 0.25001249999999997,
  'Avg. Score': 0.5,
  'Sum of Scores': 1

##### TO INVESTIGATE: Various weight factors and metrics

## STEP 4: Compare all strategy sets for a given N to determine which value of C is optimal

### 4a. Build analysis dataframe for N and an arbitrary Cmax, with metrics for each C.

In [18]:
res = pd.DataFrame(res)
res

,Credits,Strategies,Votes per Credit,Total Wasted Credits,Avg. Wasted Credits per Strategy,Avg. Min Vote Diff,Avg. Max Vote Diff,Avg. MinMax Vote Diff,Avg. Diff,Avg. Score,Sum of Scores,Avg. Weight,Sum of Weights
0,1,1,1.000000,0,0.000000,0.000000,1.000000,1.000000,0.500000,0.000000,0.000000,1.000000,1.000000
1,2,2,1.000000,1,0.500000,0.000000,0.750000,0.750000,0.458350,1.000000,2.000000,0.750000,1.500000
2,3,3,1.000000,3,1.000000,0.000000,0.583333,0.583333,0.319450,1.666667,5.000000,0.583333,1.750000
3,4,2,1.000000,3,1.500000,0.000000,0.375000,0.375000,0.208338,-0.250000,-0.500000,0.375000,0.750000
4,5,3,0.866667,5,1.666667,0.000000,0.791667,0.791667,0.458346,0.583333,1.750000,0.458333,1.375000
5,6,4,0.816667,8,2.000000,0.000000,0.796875,0.796875,0.421880,1.218750,4.875000,0.421875,1.687500
6,7,6,0.746825,15,2.500000,0.000000,0.473958,0.473958,0.244793,1.406250,8.437500,0.328125,1.968750
7,8,5,0.696190,15,3.000000,0.000000,0.281250,0.281250,0.144793,0.850000,4.250000,0.193750,0.968750
8,9,5,0.707302,15,3.000000,0.000000,0.528125,0.528125,0.299486,0.281250,1.406250,0.290625,1.453125
9,10,6,0.572751,10,1.666667,0.000000,0.958333,0.958333,0.562513,1.145833,6.875000,0.489583,2.937500


### 4b. Create additional metrics, *go back and modify strategies_for_n() metrics if applicable*

#### Test 1: Efficiency (strats/credits) * sum of scores per credit * spread (avg minmax/avg diff)

In [ ]:
res['test1'] = (res.Strategies/res.Credits) * (res['Sum of Scores']/res.Credits) * ((res['Avg. MinMax Vote Diff']/res['Avg. Diff'])/res.Credits)

In [ ]:
res

In [ ]:
res.sort_values(by = 'test1', ascending = False).head(25)

In [ ]:
res['Avg. Min Vote Diff'].describe()

In [ ]:
res['score_per_credit'] = res['Sum of Scores'] / res['Credits']
res

In [ ]:
res.sort_values(by = 'score_per_credit')

In [ ]:
res['score2'] = res.score_per_credit * (res['Strategies'] / res['Credits'])

In [ ]:
res.sort_values(by = 'score2')

In [ ]:
res[res.score_per_credit > 1].score_per_credit.nlargest(10)

In [ ]:
res['Avg. MinMax Vote Diff'].plot()

---

# To Do:
### 1. Write the function that loops through all n, and returns the optimal vote credit total for each.
### 2. Redo this with negative voting.
### 3. Explore continuous voting.

##### Discovery: The minimal number of credits which allow for strategies where no two numbers are equal (min diff > 0) is equal to the [n-1, n-2,...,0] strategy credit level.